In [66]:
from geoband.API import *
import pandas as pd
import math

# 데이터 불러오기

In [72]:
GetCompasData('SBJ_2107_004', '5', '5.112신고.csv')
GetCompasData('SBJ_2107_004', '6', '6.범죄발생원표.csv')
GetCompasData('SBJ_2107_004', '7', '7.범죄검거원표.csv')
GetCompasData('SBJ_2107_004', '12', '12.경찰서별_지구대별_인원현황.csv')
GetCompasData('SBJ_2107_004', '1', '1.5개시도_경찰서_관할경계.geojson')

In [67]:
data_112 = pd.read_csv('5.112신고.csv')

In [68]:
crime1 = pd.read_csv('6.범죄발생원표.csv')

/opt/app-root/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [69]:
crime2 = pd.read_csv('7.범죄검거원표.csv')


In [133]:
car_ac = pd.read_csv('11.5개시도_교통사고현황.csv')


In [71]:
police = pd.read_csv('12.경찰서별_지구대별_인원현황.csv')

# 112신고 데이터 전처리

In [42]:
data_112

,date,jur_stn,report_sx,inc_info
0,20180603.0,서울서부,남성,분실습득
1,20180603.0,서울송파,여성,폭력
2,20180603.0,서울강서,남성,소음
3,20180603.0,서울영등포,여성,실종(실종아동 등)
4,20180603.0,서울강서,남성,상담문의
...,...,...,...,...
9229106,20210531.0,마산동부,남성,교통사고
9229107,20210531.0,마산중부,남성,가정폭력
9229108,20210531.0,마산중부,불상,화재
9229109,20210531.0,마산동부,남성,기타형사범


In [8]:
data_112.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9229111 entries, 0 to 9229110
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   date       float64
 1   jur_stn    object 
 2   report_sx  object 
 3   inc_info   object 
dtypes: float64(1), object(3)
memory usage: 281.7+ MB


In [9]:
len(set(data_112['jur_stn']))

41

In [11]:
set(data_112['inc_info'])

{'FTX',
 nan,
 '가정폭력',
 '가출 등',
 '강도',
 '경비업체요청',
 '공갈',
 '교통불편',
 '교통사고',
 '교통위반',
 '구조요청',
 '기타_타기관',
 '기타경범',
 '기타형사범',
 '납치감금',
 '내용확인불가',
 '노점상',
 '데이트폭력',
 '도박',
 '무전취식승차',
 '변사자',
 '보호조치',
 '분실습득',
 '비상벨',
 '사기',
 '사망.대형사고',
 '살인',
 '상담문의',
 '서비스요청',
 '성폭력',
 '소음',
 '수배불심자',
 '스토킹',
 '시비',
 '실종(실종아동 등)',
 '아동학대(가정내)',
 '아동학대(기타)',
 '위험동물',
 '위험방지',
 '인피도주',
 '자살',
 '재물손괴',
 '재해재난',
 '절도',
 '주거침입',
 '주취자',
 '청소년비행',
 '청탁금지법',
 '치기',
 '폭력',
 '풍속영업',
 '학교폭력',
 '행패소란',
 '협박',
 '화재'}

In [72]:
data_112['date'] = data_112['date'].astype(str)

In [73]:
data_112['date'] = data_112['date'].str[:6]

In [74]:
sum(data_112['date']<='201706')

0

In [15]:
sum(data_112['date']<='201805') # 2018년 5월까지 데이터 없음

0

In [16]:
sum((data_112['date']>'201706')&(data_112['date']<='201712'))

0

In [17]:
sum((data_112['date']>'201712')&(data_112['date']<='201806'))

278927

In [18]:
sum((data_112['date']>'201806')&(data_112['date']<='201812'))

1566665

In [19]:
sum((data_112['date']>'201812')&(data_112['date']<='201906'))

1536091

In [20]:
sum((data_112['date']>'201906')&(data_112['date']<='201912'))

1707102

In [21]:
sum((data_112['date']>'201912')&(data_112['date']<='202006'))

1459963

In [22]:
sum((data_112['date']>'202006')&(data_112['date']<='202012'))

1531866

In [75]:
data_112.loc[(data_112['date']>'201712')&(data_112['date']<='201806'),'year'] = '2018상'
data_112.loc[(data_112['date']>'201806')&(data_112['date']<='201812'),'year'] = '2018하'
data_112.loc[(data_112['date']>'201812')&(data_112['date']<='201906'),'year'] = '2019상'
data_112.loc[(data_112['date']>'201906')&(data_112['date']<='201912'),'year'] = '2019하'
data_112.loc[(data_112['date']>'201912')&(data_112['date']<='202012'),'year'] = '2020'
data_112.loc[(data_112['date']>'202012')&(data_112['date']<='202106'),'year'] = '2021상'

In [76]:
data_112

,date,jur_stn,report_sx,inc_info,year
0,201806,서울서부,남성,분실습득,2018상
1,201806,서울송파,여성,폭력,2018상
2,201806,서울강서,남성,소음,2018상
3,201806,서울영등포,여성,실종(실종아동 등),2018상
4,201806,서울강서,남성,상담문의,2018상
...,...,...,...,...,...
9229106,202105,마산동부,남성,교통사고,2021상
9229107,202105,마산중부,남성,가정폭력,2021상
9229108,202105,마산중부,불상,화재,2021상
9229109,202105,마산동부,남성,기타형사범,2021상


In [77]:
data_112.loc[data_112['inc_info'].str.contains('폭력')==True,'112신고_폭력'] = 1

In [78]:
data_112.loc[data_112['inc_info'] == '절도','112신고_절도'] = 1

In [79]:
data_112.loc[data_112['inc_info'] == '강도','112신고_강도'] = 1

In [80]:
data_112.loc[data_112['inc_info'] == '살인','112신고_살인'] = 1

In [81]:
data_112.loc[data_112['inc_info'] == '교통사고','112신고_교통사고'] = 1

In [82]:
data_112.loc[data_112['inc_info'] == '교통불편','112신고_교통불편'] = 1

In [83]:
data_112.loc[data_112['inc_info'] == '교통위반','112신고_교통위반'] = 1

In [84]:
data_112['jur_stn'] = data_112['jur_stn'] + '경찰서'

In [85]:
police_list = set(data_112['jur_stn'])

In [86]:
data_112 = data_112.groupby(['jur_stn','year']).sum()
# data_112 = data_112[['절도/폭력','강도/살인']].astype(int)
data_112 = data_112.astype(int)
data_112

112신고_폭력  112신고_절도  112신고_강도  112신고_살인  112신고_교통사고  112신고_교통불편  \
jur_stn year                                                                    
마산동부경찰서 2018상       261        71         0         0         275         213   
        2018하      1547       436         0         2        1752        1132   
        2019상      1751       521         1         0        2007        1248   
        2019하      1715       546         5         3        1910        1190   
        2020       2941      1039         3         1        3751        2011   
...                 ...       ...       ...       ...         ...         ...   
창원중부경찰서 2018하      1719       572         2         0        2334         863   
        2019상      1883       662         1         0        2697        1059   
        2019하      1830       645         0         1        2819        1187   
        2020       3005      1265         1         2        5126        1834   
        2021상      1256       521         1         0        2100         686   

               112신고_교통위반  
jur_stn year               
마산동부경찰서 2018상          27  
        2018하         242  
        2019상         339  
        2019하         433  
        2020          742  
...                   ...  
창원중부경찰서 2018하         322  
        2019상         491  
        2019하         614  
        2020          641  
        2021상         240  

[244 rows x 7 columns]

In [88]:
data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_절도'] = round(data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_절도']/6)
data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_폭력'] = round(data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_폭력']/6)
data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_강도'] = round(data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_강도']/6)
data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_살인'] = round(data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_살인']/6)
data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_교통사고'] = round(data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_교통사고']/6)
data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_교통불편'] = round(data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_교통불편']/6)
data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_교통위반'] = round(data_112.loc[(slice(None),['2018하','2019상','2019하','2021상']),'112신고_교통위반']/6)
data_112.loc[(slice(None),['2020']),'112신고_절도'] = round(data_112.loc[(slice(None),['2020']),'112신고_절도']/12)
data_112.loc[(slice(None),['2020']),'112신고_폭력'] = round(data_112.loc[(slice(None),['2020']),'112신고_폭력']/12)
data_112.loc[(slice(None),['2020']),'112신고_강도'] = round(data_112.loc[(slice(None),['2020']),'112신고_강도']/12)
data_112.loc[(slice(None),['2020']),'112신고_살인'] = round(data_112.loc[(slice(None),['2020']),'112신고_살인']/12)
data_112.loc[(slice(None),['2020']),'112신고_교통사고'] = round(data_112.loc[(slice(None),['2020']),'112신고_교통사고']/12)
data_112.loc[(slice(None),['2020']),'112신고_교통불편'] = round(data_112.loc[(slice(None),['2020']),'112신고_교통불편']/12)
data_112.loc[(slice(None),['2020']),'112신고_교통위반'] = round(data_112.loc[(slice(None),['2020']),'112신고_교통위반']/12)

In [92]:
data_112

112신고_폭력  112신고_절도  112신고_강도  112신고_살인  112신고_교통사고  112신고_교통불편  \
jur_stn year                                                                    
마산동부경찰서 2018상     261.0      71.0       0.0       0.0       275.0       213.0   
        2018하     258.0      73.0       0.0       0.0       292.0       189.0   
        2019상     292.0      87.0       0.0       0.0       334.0       208.0   
        2019하     286.0      91.0       1.0       0.0       318.0       198.0   
        2020      245.0      87.0       0.0       0.0       313.0       168.0   
...                 ...       ...       ...       ...         ...         ...   
창원중부경찰서 2018하     286.0      95.0       0.0       0.0       389.0       144.0   
        2019상     314.0     110.0       0.0       0.0       450.0       176.0   
        2019하     305.0     108.0       0.0       0.0       470.0       198.0   
        2020      250.0     105.0       0.0       0.0       427.0       153.0   
        2021상     209.0      87.0       0.0       0.0       350.0       114.0   

               112신고_교통위반  
jur_stn year               
마산동부경찰서 2018상        27.0  
        2018하        40.0  
        2019상        56.0  
        2019하        72.0  
        2020         62.0  
...                   ...  
창원중부경찰서 2018하        54.0  
        2019상        82.0  
        2019하       102.0  
        2020         53.0  
        2021상        40.0  

[244 rows x 7 columns]

In [93]:
data_112.loc[('세종경찰서',slice(None))]

,112신고_폭력,112신고_절도,112신고_강도,112신고_살인,112신고_교통사고,112신고_교통불편,112신고_교통위반
year,,,,,,,
2018상,94.0,48.0,0.0,0.0,179.0,51.0,25.0
2018하,126.0,47.0,0.0,0.0,220.0,65.0,37.0
2019상,227.0,106.0,0.0,0.0,404.0,116.0,68.0
2021상,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
data_112.loc[('세종경찰서','2019하'),['112신고_폭력','112신고_절도','112신고_강도','112신고_살인','112신고_교통사고','112신고_교통불편','112신고_교통위반']] = None
data_112.loc[('세종경찰서','2020'),['112신고_폭력','112신고_절도','112신고_강도','112신고_살인','112신고_교통사고','112신고_교통불편','112신고_교통위반']] = None
data_112.loc[('세종경찰서','2021상'),['112신고_폭력','112신고_절도','112신고_강도','112신고_살인','112신고_교통사고','112신고_교통불편','112신고_교통위반']] = None

In [95]:
for x in police_list:
    data_112.loc[(x,'2017상'),['112신고_폭력','112신고_절도','112신고_강도','112신고_살인','112신고_교통사고','112신고_교통불편','112신고_교통위반']] = None
    data_112.loc[(x,'2017하'),['112신고_폭력','112신고_절도','112신고_강도','112신고_살인','112신고_교통사고','112신고_교통불편','112신고_교통위반']] = None

data_112

112신고_폭력  112신고_절도  112신고_강도  112신고_살인  112신고_교통사고  112신고_교통불편  \
jur_stn year                                                                    
마산동부경찰서 2018상     261.0      71.0       0.0       0.0       275.0       213.0   
        2018하     258.0      73.0       0.0       0.0       292.0       189.0   
        2019상     292.0      87.0       0.0       0.0       334.0       208.0   
        2019하     286.0      91.0       1.0       0.0       318.0       198.0   
        2020      245.0      87.0       0.0       0.0       313.0       168.0   
...                 ...       ...       ...       ...         ...         ...   
서울수서경찰서 2017하       NaN       NaN       NaN       NaN         NaN         NaN   
서울마포경찰서 2017상       NaN       NaN       NaN       NaN         NaN         NaN   
        2017하       NaN       NaN       NaN       NaN         NaN         NaN   
서울양천경찰서 2017상       NaN       NaN       NaN       NaN         NaN         NaN   
        2017하       NaN       NaN       NaN       NaN         NaN         NaN   

               112신고_교통위반  
jur_stn year               
마산동부경찰서 2018상        27.0  
        2018하        40.0  
        2019상        56.0  
        2019하        72.0  
        2020         62.0  
...                   ...  
서울수서경찰서 2017하         NaN  
서울마포경찰서 2017상         NaN  
        2017하         NaN  
서울양천경찰서 2017상         NaN  
        2017하         NaN  

[328 rows x 7 columns]

In [96]:
data_112 = data_112.sort_index(axis=0)

In [97]:
data_112

112신고_폭력  112신고_절도  112신고_강도  112신고_살인  112신고_교통사고  112신고_교통불편  \
jur_stn year                                                                    
마산동부경찰서 2017상       NaN       NaN       NaN       NaN         NaN         NaN   
        2017하       NaN       NaN       NaN       NaN         NaN         NaN   
        2018상     261.0      71.0       0.0       0.0       275.0       213.0   
        2018하     258.0      73.0       0.0       0.0       292.0       189.0   
        2019상     292.0      87.0       0.0       0.0       334.0       208.0   
...                 ...       ...       ...       ...         ...         ...   
창원중부경찰서 2018하     286.0      95.0       0.0       0.0       389.0       144.0   
        2019상     314.0     110.0       0.0       0.0       450.0       176.0   
        2019하     305.0     108.0       0.0       0.0       470.0       198.0   
        2020      250.0     105.0       0.0       0.0       427.0       153.0   
        2021상     209.0      87.0       0.0       0.0       350.0       114.0   

               112신고_교통위반  
jur_stn year               
마산동부경찰서 2017상         NaN  
        2017하         NaN  
        2018상        27.0  
        2018하        40.0  
        2019상        56.0  
...                   ...  
창원중부경찰서 2018하        54.0  
        2019상        82.0  
        2019하       102.0  
        2020         53.0  
        2021상        40.0  

[328 rows x 7 columns]

In [98]:
for x in police_list:
    data_112.loc[(x,'2017상'),['112신고_폭력','112신고_절도','112신고_강도','112신고_살인','112신고_교통사고','112신고_교통불편','112신고_교통위반']] = data_112.loc[(x,slice(None))].interpolate(method ='linear',limit_direction ='both').loc['2017상',:]
    data_112.loc[(x,'2017하'),['112신고_폭력','112신고_절도','112신고_강도','112신고_살인','112신고_교통사고','112신고_교통불편','112신고_교통위반']] = data_112.loc[(x,slice(None))].interpolate(method ='linear',limit_direction ='both').loc['2017하',:]
    
data_112.loc[('세종경찰서','2019하'),['112신고_폭력','112신고_절도','112신고_강도','112신고_살인','112신고_교통사고','112신고_교통불편','112신고_교통위반']] = data_112.loc[(x,slice(None))].interpolate(method ='linear',limit_direction ='both').loc['2019하',:]    
data_112.loc[('세종경찰서','2020'),['112신고_폭력','112신고_절도','112신고_강도','112신고_살인','112신고_교통사고','112신고_교통불편','112신고_교통위반']] = data_112.loc[(x,slice(None))].interpolate(method ='linear',limit_direction ='both').loc['2020',:]
data_112.loc[('세종경찰서','2021상'),['112신고_폭력','112신고_절도','112신고_강도','112신고_살인','112신고_교통사고','112신고_교통불편','112신고_교통위반']] = data_112.loc[(x,slice(None))].interpolate(method ='linear',limit_direction ='both').loc['2021상',:]
data_112

112신고_폭력  112신고_절도  112신고_강도  112신고_살인  112신고_교통사고  112신고_교통불편  \
jur_stn year                                                                    
마산동부경찰서 2017상     261.0      71.0       0.0       0.0       275.0       213.0   
        2017하     261.0      71.0       0.0       0.0       275.0       213.0   
        2018상     261.0      71.0       0.0       0.0       275.0       213.0   
        2018하     258.0      73.0       0.0       0.0       292.0       189.0   
        2019상     292.0      87.0       0.0       0.0       334.0       208.0   
...                 ...       ...       ...       ...         ...         ...   
창원중부경찰서 2018하     286.0      95.0       0.0       0.0       389.0       144.0   
        2019상     314.0     110.0       0.0       0.0       450.0       176.0   
        2019하     305.0     108.0       0.0       0.0       470.0       198.0   
        2020      250.0     105.0       0.0       0.0       427.0       153.0   
        2021상     209.0      87.0       0.0       0.0       350.0       114.0   

               112신고_교통위반  
jur_stn year               
마산동부경찰서 2017상        27.0  
        2017하        27.0  
        2018상        27.0  
        2018하        40.0  
        2019상        56.0  
...                   ...  
창원중부경찰서 2018하        54.0  
        2019상        82.0  
        2019하       102.0  
        2020         53.0  
        2021상        40.0  

[328 rows x 7 columns]

In [100]:
data_112 = data_112.astype(int)

In [101]:
data_112

112신고_폭력  112신고_절도  112신고_강도  112신고_살인  112신고_교통사고  112신고_교통불편  \
jur_stn year                                                                    
마산동부경찰서 2017상       261        71         0         0         275         213   
        2017하       261        71         0         0         275         213   
        2018상       261        71         0         0         275         213   
        2018하       258        73         0         0         292         189   
        2019상       292        87         0         0         334         208   
...                 ...       ...       ...       ...         ...         ...   
창원중부경찰서 2018하       286        95         0         0         389         144   
        2019상       314       110         0         0         450         176   
        2019하       305       108         0         0         470         198   
        2020        250       105         0         0         427         153   
        2021상       209        87         0         0         350         114   

               112신고_교통위반  
jur_stn year               
마산동부경찰서 2017상          27  
        2017하          27  
        2018상          27  
        2018하          40  
        2019상          56  
...                   ...  
창원중부경찰서 2018하          54  
        2019상          82  
        2019하         102  
        2020           53  
        2021상          40  

[328 rows x 7 columns]

# 범죄발생 데이터 전처리

In [67]:
crime1

,jur_stn,crm,crm_wthr,crm_clue,vic_sx,vic_age,crm_loc,crm_tm,crm_date,5m_crm_yn
0,서울수서경찰서,위조외국통화행사,미상,진정,불상,,은행,09:00-11:59,20170101,
1,서울영등포경찰서,도로교통법위반,맑음,타인신고,불상,,노상,21:00-23:59,20170101,
2,서울양천경찰서,209015100,미상,피해자신고,남자,60세초과,노상,미상,20170101,
3,서울서초경찰서,폭행,미상,피해자신고,여자,40세이하,기타,21:00-23:59,20170101,폭력
4,서울동대문경찰서,사기,미상,진정,여자,30세이하,기타,미상,20170101,
...,...,...,...,...,...,...,...,...,...,...
1073217,경남진해경찰서,사기,미상,고소,2,60세이하,기타,미상,9/27/2019,
1073218,경남진해경찰서,폭행,맑음,피해자신고,3,,차안,00:00∼02:59,10/4/2019,폭력
1073219,경남마산동부경찰서,재물손괴,미상,피해자신고,2,60세이하,주택,12:00∼14:59,9/17/2019,폭력
1073220,경남마산중부경찰서,사기,미상,진정,1,50세이하,기타,12:00∼14:59,9/14/2019,


In [82]:
set(crime1['crm'])

{' 한국마사회법위반(도박등)방조',
 ' 특수협박',
 '209014900',
 ' 유기치사',
 ' 현존건조물방화예비',
 ' 특정범죄가중처벌등에관한법률위반(관세)',
 ' 폭행치사',
 ' 성매매알선등행위의처벌에관한법률위반(성매매알선등)교사',
 ' 도로교통법위반(사고후미조치)방조',
 ' 공인회계사법위반',
 ' 특수전자기록등손괴',
 ' 액화석유가스의안전관리및사업법위반',
 ' 허위공문서작성',
 ' 아동복지법위반',
 ' 주거침입',
 ' 일반교통방해치상',
 ' 준강도미수',
 ' 실화',
 ' 협동조합기본법위반',
 ' 내란선전',
 ' 성폭력범죄의처벌등에관한특례법위반(장애인준강제추행)',
 ' 청소년기본법위반',
 ' 부정청탁및금품등수수의금지에관한법률위반',
 ' 전파법위반',
 ' 행정사법위반',
 ' 협박방조',
 ' 성폭력범죄의처벌등에관한특례법위반(통신매체이용음란)',
 ' 공무상표시손상',
 ' 특정범죄가중처벌등에관한법률위반(향정)',
 ' 도로교통법위반(무면허운전)교사',
 ' 외국환관리법위반',
 ' 위조통화지정행사방조',
 ' 도박방조',
 ' 폭행치상',
 ' 가축분뇨의관리및이용에관한법률위반',
 ' 존속폭행치상',
 ' 성폭력범죄의처벌및피해자보호등에관한법률위반(강간등치상)',
 ' 현주건조물방화치사',
 ' 건조물침입미수',
 ' 성폭력범죄의처벌등에관한특례법위반(특수강간)',
 ' 정보통신공사업법위반',
 ' 준유사강간미수',
 ' 대기환경보전법위반',
 ' 공갈미수',
 ' 위조외국통화지정행사',
 ' 문화재수리등에관한법률위반',
 ' 전자기록등내용탐지',
 ' 국가공무원법위반',
 ' 성폭력범죄의처벌등에관한특례법위반(친족관계에의한준강제추행)',
 ' 공무원자격사칭',
 ' 강도예비',
 ' 불실기재공정증서원본행사',
 ' 업무상과실자동차전복',
 ' 통화위조',
 ' 특정범죄가중처벌등에관한법률위반(보복범죄등)',
 ' 배임방조',
 ' 한부모가족지원법위반',
 ' 공문서변조교사',
 ' 특정범죄가중처벌등에관한법률위반(알선수재

In [83]:
crime1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1073222 entries, 0 to 1073221
Data columns (total 10 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   jur_stn    1073222 non-null  object
 1   crm        1073222 non-null  object
 2   crm_wthr   1073222 non-null  object
 3   crm_clue   1073222 non-null  object
 4   vic_sx     1073222 non-null  object
 5   vic_age    1073222 non-null  object
 6   crm_loc    1073222 non-null  object
 7   crm_tm     1073222 non-null  object
 8   crm_date   1073222 non-null  object
 9   5m_crm_yn  1073222 non-null  object
dtypes: object(10)
memory usage: 81.9+ MB


In [102]:
# 경기,경남,충남 제거/ 공백제거
crime1['jur_stn'] = crime1['jur_stn'].str.strip()
crime1['jur_stn'] = crime1['jur_stn'].str.replace('경기','')
crime1['jur_stn'] = crime1['jur_stn'].str.replace('경남','')
crime1['jur_stn'] = crime1['jur_stn'].str.replace('충남','')
len(set(crime1['jur_stn']))

41

In [103]:
import datetime

In [104]:
crime1['crm_date'] = list(map(lambda x:str(datetime.datetime.strptime(x,"%m/%d/%Y").year)+str(datetime.datetime.strptime(x,"%m/%d/%Y").month) if '/' in str(x) else str(x)[:6],crime1['crm_date']))

In [105]:
crime1['crm_date'] = list(map(lambda x: x[:4]+'0'+x[4] if len(x)==5 else x,crime1['crm_date']))

In [108]:
sum(crime1['crm_date']>='202001') # 2020년 데이터 없음

0

In [106]:
crime1.loc[(crime1['crm_date']>'201612')&(crime1['crm_date']<='201706'),'year'] = '2017상'
crime1.loc[(crime1['crm_date']>'201706')&(crime1['crm_date']<='201712'),'year'] = '2017하'
crime1.loc[(crime1['crm_date']>'201712')&(crime1['crm_date']<='201806'),'year'] = '2018상'
crime1.loc[(crime1['crm_date']>'201806')&(crime1['crm_date']<='201812'),'year'] = '2018하'
crime1.loc[(crime1['crm_date']>'201812')&(crime1['crm_date']<='201906'),'year'] = '2019상'
crime1.loc[(crime1['crm_date']>'201906')&(crime1['crm_date']<='201912'),'year'] = '2019하'


In [107]:
set(crime1['5m_crm_yn'])

{' ', '강간 및 강제추행', '강도', '살인', '절도', '폭력'}

In [108]:
crime1['crm'] = crime1['crm'].str.strip()

In [109]:
crm1 = set(crime1.loc[crime1['5m_crm_yn'] == '강간 및 강제추행','crm'])
crm2 = set(crime1.loc[crime1['5m_crm_yn'] == '절도','crm'])
crm3 = set(crime1.loc[crime1['5m_crm_yn'] == '폭력','crm'])
crm4 = set(crime1.loc[crime1['5m_crm_yn'] == '강도','crm'])
crm5 = set(crime1.loc[crime1['5m_crm_yn'] == '살인','crm'])

In [110]:
# crime1.loc[(crime1['5m_crm_yn'] == '절도')|(crime1['5m_crm_yn'] == '폭력'),'범죄발생_절도/폭력'] = 1
# crime1.loc[(crime1['5m_crm_yn'] == '강도')|(crime1['5m_crm_yn'] == '살인'),'범죄발생_강도/살인'] = 1
# crime1.loc[crime1['5m_crm_yn'] == '강간 및 강제추행','범죄발생_강간/강제추행'] = 1

In [111]:
crime1.loc[(crime1['5m_crm_yn'] == '절도'),'범죄발생_절도'] = 1
crime1.loc[(crime1['5m_crm_yn'] == '폭력'),'범죄발생_폭력'] = 1
crime1.loc[(crime1['5m_crm_yn'] == '강도'),'범죄발생_강도'] = 1
crime1.loc[(crime1['5m_crm_yn'] == '살인'),'범죄발생_살인'] = 1
crime1.loc[crime1['5m_crm_yn'] == '강간 및 강제추행','범죄발생_강간/강제추행'] = 1

In [112]:
crime1

,jur_stn,crm,crm_wthr,crm_clue,vic_sx,vic_age,crm_loc,crm_tm,crm_date,5m_crm_yn,year,범죄발생_절도,범죄발생_폭력,범죄발생_강도,범죄발생_살인,범죄발생_강간/강제추행
0,서울수서경찰서,위조외국통화행사,미상,진정,불상,,은행,09:00-11:59,201701,,2017상,NaN,NaN,NaN,NaN,NaN
1,서울영등포경찰서,도로교통법위반,맑음,타인신고,불상,,노상,21:00-23:59,201701,,2017상,NaN,NaN,NaN,NaN,NaN
2,서울양천경찰서,209015100,미상,피해자신고,남자,60세초과,노상,미상,201701,,2017상,NaN,NaN,NaN,NaN,NaN
3,서울서초경찰서,폭행,미상,피해자신고,여자,40세이하,기타,21:00-23:59,201701,폭력,2017상,NaN,1.0,NaN,NaN,NaN
4,서울동대문경찰서,사기,미상,진정,여자,30세이하,기타,미상,201701,,2017상,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073217,진해경찰서,사기,미상,고소,2,60세이하,기타,미상,201909,,2019하,NaN,NaN,NaN,NaN,NaN
1073218,진해경찰서,폭행,맑음,피해자신고,3,,차안,00:00∼02:59,201910,폭력,2019하,NaN,1.0,NaN,NaN,NaN
1073219,마산동부경찰서,재물손괴,미상,피해자신고,2,60세이하,주택,12:00∼14:59,201909,폭력,2019하,NaN,1.0,NaN,NaN,NaN
1073220,마산중부경찰서,사기,미상,진정,1,50세이하,기타,12:00∼14:59,201909,,2019하,NaN,NaN,NaN,NaN,NaN


In [113]:
police_list = set(crime1['jur_stn'])

In [114]:
crime1 = crime1.groupby(['jur_stn','year']).sum()
# crime1 = crime1[['범죄발생_절도/폭력','범죄발생_강도/살인','범죄발생_강간/강제추행']].astype(int)
crime1 = crime1.astype(int)
crime1

범죄발생_절도  범죄발생_폭력  범죄발생_강도  범죄발생_살인  범죄발생_강간/강제추행
jur_stn year                                                   
마산동부경찰서 2017상      367      547        2        2            30
        2017하      430      610        2        1            34
        2018상      340      541        0        1            30
        2018하      429      640        0        2            38
        2019상      404      580        0        2            44
...                ...      ...      ...      ...           ...
창원중부경찰서 2017하      493      672        1        0            64
        2018상      475      669        5        2            45
        2018하      495      719        5        1            39
        2019상      560      668        3        1            46
        2019하      296      526        0        1            44

[246 rows x 5 columns]

In [115]:
crm_list = ['범죄발생_절도','범죄발생_폭력','범죄발생_강도','범죄발생_살인','범죄발생_강간/강제추행']

for x in crm_list:
    crime1.loc[(slice(None),['2017상','2017하','2018상','2018하','2019상','2019하']),x] = round(crime1.loc[(slice(None),['2017상','2017하','2018상','2018하','2019상','2019하']),x]/6)

In [116]:
for x in police_list:
    crime1.loc[(x,'2020'),['범죄발생_절도','범죄발생_폭력','범죄발생_강도','범죄발생_살인','범죄발생_강간/강제추행']] = None


crime1

범죄발생_절도  범죄발생_폭력  범죄발생_강도  범죄발생_살인  범죄발생_강간/강제추행
jur_stn  year                                                   
마산동부경찰서  2017상     61.0     91.0      0.0      0.0           5.0
         2017하     72.0    102.0      0.0      0.0           6.0
         2018상     57.0     90.0      0.0      0.0           5.0
         2018하     72.0    107.0      0.0      0.0           6.0
         2019상     67.0     97.0      0.0      0.0           7.0
...                 ...      ...      ...      ...           ...
서울광진경찰서  2020       NaN      NaN      NaN      NaN           NaN
서울마포경찰서  2020       NaN      NaN      NaN      NaN           NaN
서울수서경찰서  2020       NaN      NaN      NaN      NaN           NaN
서울남대문경찰서 2020       NaN      NaN      NaN      NaN           NaN
서울양천경찰서  2020       NaN      NaN      NaN      NaN           NaN

[287 rows x 5 columns]

In [117]:
crime1 = crime1.sort_index(axis=0)

In [118]:
for x in police_list:
    crime1.loc[(x,'2020'),['범죄발생_절도','범죄발생_폭력','범죄발생_강도','범죄발생_살인','범죄발생_강간/강제추행']] = crime1.loc[(x,slice(None))].interpolate(method ='linear',limit_direction ='both').loc['2020',:]

In [119]:
crime1

범죄발생_절도  범죄발생_폭력  범죄발생_강도  범죄발생_살인  범죄발생_강간/강제추행
jur_stn year                                                   
마산동부경찰서 2017상     61.0     91.0      0.0      0.0           5.0
        2017하     72.0    102.0      0.0      0.0           6.0
        2018상     57.0     90.0      0.0      0.0           5.0
        2018하     72.0    107.0      0.0      0.0           6.0
        2019상     67.0     97.0      0.0      0.0           7.0
...                ...      ...      ...      ...           ...
창원중부경찰서 2018상     79.0    112.0      1.0      0.0           8.0
        2018하     82.0    120.0      1.0      0.0           6.0
        2019상     93.0    111.0      0.0      0.0           8.0
        2019하     49.0     88.0      0.0      0.0           7.0
        2020      49.0     88.0      0.0      0.0           7.0

[287 rows x 5 columns]

# 범죄검거 데이터 전처리

In [123]:
crime2

,date,jur_stn,crm
0,201701,경남마산중부경찰서,사기
1,201701,경남마산중부경찰서,경범죄처벌법위반
2,201612,경남마산중부경찰서,도로교통법위반(음주운전)
3,201702,경남마산중부경찰서,위계공무집행방해
4,201702,경남마산중부경찰서,위계공무집행방해
...,...,...,...
928560,201912,경남창원중부경찰서,교통사고처리특례법위반(치상)
928561,201912,경남창원중부경찰서,사기
928562,201912,경남창원중부경찰서,사기
928563,201912,경남창원중부경찰서,사기


In [124]:
crime2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928565 entries, 0 to 928564
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   date     928565 non-null  int64 
 1   jur_stn  928565 non-null  object
 2   crm      928565 non-null  object
dtypes: int64(1), object(2)
memory usage: 21.3+ MB


In [120]:
# 경기,경남,충남 제거/ 공백제거
crime2['jur_stn'] = crime2['jur_stn'].str.strip()
crime2['jur_stn'] = crime2['jur_stn'].str.replace('경기','')
crime2['jur_stn'] = crime2['jur_stn'].str.replace('경남','')
crime2['jur_stn'] = crime2['jur_stn'].str.replace('충남','')
len(set(crime2['jur_stn']))

41

In [121]:
crime2.loc[crime2['crm'].isin(crm1),'범죄검거_강간/강제추행'] = 1
crime2.loc[crime2['crm'].isin(crm2),'범죄검거_절도'] = 1
crime2.loc[crime2['crm'].isin(crm3),'범죄검거_폭력'] = 1
crime2.loc[crime2['crm'].isin(crm4),'범죄검거_강도'] = 1
crime2.loc[crime2['crm'].isin(crm5),'범죄검거_살인'] = 1

In [122]:
crime2['date'] = crime2['date'].astype(str)

In [128]:
sum(crime2['date']>='202001') # 2020년 데이터 없음

0

In [123]:
crime2.loc[(crime2['date']>'201612')&(crime2['date']<='201706'),'year'] = '2017상'
crime2.loc[(crime2['date']>'201706')&(crime2['date']<='201712'),'year'] = '2017하'
crime2.loc[(crime2['date']>'201712')&(crime2['date']<='201806'),'year'] = '2018상'
crime2.loc[(crime2['date']>'201806')&(crime2['date']<='201812'),'year'] = '2018하'
crime2.loc[(crime2['date']>'201812')&(crime2['date']<='201906'),'year'] = '2019상'
crime2.loc[(crime2['date']>'201906')&(crime2['date']<='201912'),'year'] = '2019하'


In [124]:
crime2 = crime2.groupby(['jur_stn','year']).sum()
# crime2 = crime2[['범죄검거_절도/폭력','범죄검거_강도/살인','범죄검거_강간/강제추행']].astype(int)
crime2 = crime2.astype(int)
crime2

범죄검거_강간/강제추행  범죄검거_절도  범죄검거_폭력  범죄검거_강도  범죄검거_살인
jur_stn year                                                   
마산동부경찰서 2017상            32      215      489        1        1
        2017하            29      214      491        1        1
        2018상            23      243      465        1        1
        2018하            36      198      556        0        2
        2019상            34      208      492        0        3
...                     ...      ...      ...      ...      ...
창원중부경찰서 2017하            65      330      569        1        0
        2018상            38      334      644        4        2
        2018하            39      315      616        6        1
        2019상            43      404      593        2        0
        2019하            67      397      655        1        2

[246 rows x 5 columns]

In [126]:
crm2_list = ['범죄검거_절도','범죄검거_폭력','범죄검거_강도','범죄검거_살인','범죄검거_강간/강제추행']

for x in crm2_list:
    crime2.loc[(slice(None),['2017상','2017하','2018상','2018하','2019상','2019하']),x] = round(crime2.loc[(slice(None),['2017상','2017하','2018상','2018하','2019상','2019하']),x]/6)
    

In [127]:
for x in police_list:
    crime2.loc[(x,'2020'),['범죄검거_절도','범죄검거_폭력','범죄검거_강도','범죄검거_살인','범죄검거_강간/강제추행']] = None

crime2

범죄검거_강간/강제추행  범죄검거_절도  범죄검거_폭력  범죄검거_강도  범죄검거_살인
jur_stn  year                                                   
마산동부경찰서  2017상           5.0     36.0     82.0      0.0      0.0
         2017하           5.0     36.0     82.0      0.0      0.0
         2018상           4.0     40.0     78.0      0.0      0.0
         2018하           6.0     33.0     93.0      0.0      0.0
         2019상           6.0     35.0     82.0      0.0      0.0
...                      ...      ...      ...      ...      ...
서울광진경찰서  2020            NaN      NaN      NaN      NaN      NaN
서울마포경찰서  2020            NaN      NaN      NaN      NaN      NaN
서울수서경찰서  2020            NaN      NaN      NaN      NaN      NaN
서울남대문경찰서 2020            NaN      NaN      NaN      NaN      NaN
서울양천경찰서  2020            NaN      NaN      NaN      NaN      NaN

[287 rows x 5 columns]

In [128]:
crime2 = crime2.sort_index(axis=0)

In [129]:
for x in police_list:
    crime2.loc[(x,'2020'),['범죄검거_절도','범죄검거_폭력','범죄검거_강도','범죄검거_살인','범죄검거_강간/강제추행']] = crime2.loc[(x,slice(None))].interpolate(method ='linear',limit_direction ='both').loc['2020',:]
    

In [130]:
crime2 = crime2.astype(int)

In [131]:
crime2

범죄검거_강간/강제추행  범죄검거_절도  범죄검거_폭력  범죄검거_강도  범죄검거_살인
jur_stn year                                                   
마산동부경찰서 2017상             5       36       82        0        0
        2017하             5       36       82        0        0
        2018상             4       40       78        0        0
        2018하             6       33       93        0        0
        2019상             6       35       82        0        0
...                     ...      ...      ...      ...      ...
창원중부경찰서 2018상             6       56      107        1        0
        2018하             6       52      103        1        0
        2019상             7       67       99        0        0
        2019하            11       66      109        0        0
        2020             11       66      109        0        0

[287 rows x 5 columns]

# 교통사고 데이터 전처리

In [134]:
car_ac

,date,acc_case,ofn_sx,ofn_age,vic_sx,vic_age,jur_stn
0,2018년 1월,차대차 - 측면충돌,남,20대,남,50대,서울중부
1,2018년 1월,차대차 - 측면충돌,여,50대,남,70대,서울중부
2,2018년 1월,차대차 - 측면충돌,남,40대,남,50대,서울중부
3,2018년 1월,차대차 - 기타,기타불명,미분류,여,40대,서울중부
4,2018년 1월,차대차 - 기타,남,60대,남,50대,서울중부
...,...,...,...,...,...,...,...
140949,2019년 8월,차대사람 - 횡단중,남,40대,여,90대 이상,마산중부
140950,2020년 12월,차대사람 - 차도통행중,여,40대,여,90대 이상,창원중부
140951,2019년 4월,차대사람 - 횡단중,남,60대,남,90대 이상,진해경찰서
140952,2019년 5월,차대사람 - 차도통행중,남,20대,남,90대 이상,진해경찰서


In [135]:
car_ac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140954 entries, 0 to 140953
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      140954 non-null  object
 1   acc_case  140954 non-null  object
 2   ofn_sx    140954 non-null  object
 3   ofn_age   140954 non-null  object
 4   vic_sx    136527 non-null  object
 5   vic_age   140954 non-null  object
 6   jur_stn   140954 non-null  object
dtypes: object(7)
memory usage: 7.5+ MB


In [136]:
set(car_ac['jur_stn'])

{'마산동부',
 '마산중부',
 '서울강남',
 '서울강동',
 '서울강북',
 '서울강서',
 '서울관악',
 '서울광진',
 '서울구로',
 '서울금천',
 '서울남대문',
 '서울노원',
 '서울도봉',
 '서울동대문',
 '서울동작',
 '서울마포',
 '서울방배',
 '서울서대문',
 '서울서부',
 '서울서초',
 '서울성동',
 '서울성북',
 '서울송파',
 '서울수서',
 '서울양천',
 '서울영등포',
 '서울용산',
 '서울은평',
 '서울종로',
 '서울종암',
 '서울중랑',
 '서울중부',
 '서울혜화',
 '세종',
 '수원남부',
 '수원서부',
 '수원중부',
 '진주',
 '진해경찰서',
 '창원서부',
 '창원중부'}

In [137]:
car_ac['jur_stn'] = car_ac['jur_stn'].str.replace('경찰서','')
car_ac['jur_stn'] = car_ac['jur_stn'] + '경찰서'

In [138]:
car_ac['date'] = car_ac['date'].str[:4] + car_ac['date'].str[6:-1]
car_ac['date'] = list(map(lambda x: x[:4]+'0'+x[4] if len(x)==5 else x,car_ac['date']))

In [105]:
sum(car_ac['date']<='201712') #2017년 데이터 없음 

0

In [139]:
car_ac.loc[(car_ac['date']>'201612')&(car_ac['date']<='201706'),'year'] = '2017상'
car_ac.loc[(car_ac['date']>'201706')&(car_ac['date']<='201712'),'year'] = '2017하'
car_ac.loc[(car_ac['date']>'201712')&(car_ac['date']<='201806'),'year'] = '2018상'
car_ac.loc[(car_ac['date']>'201806')&(car_ac['date']<='201812'),'year'] = '2018하'
car_ac.loc[(car_ac['date']>'201812')&(car_ac['date']<='201906'),'year'] = '2019상'
car_ac.loc[(car_ac['date']>'201906')&(car_ac['date']<='201912'),'year'] = '2019하'
car_ac.loc[(car_ac['date']>'201912')&(car_ac['date']<='202012'),'year'] = '2020'
car_ac.loc[(car_ac['date']>'202012')&(car_ac['date']<='202106'),'year'] = '2021상'

In [140]:
car_ac

,date,acc_case,ofn_sx,ofn_age,vic_sx,vic_age,jur_stn,year
0,201801,차대차 - 측면충돌,남,20대,남,50대,서울중부경찰서,2018상
1,201801,차대차 - 측면충돌,여,50대,남,70대,서울중부경찰서,2018상
2,201801,차대차 - 측면충돌,남,40대,남,50대,서울중부경찰서,2018상
3,201801,차대차 - 기타,기타불명,미분류,여,40대,서울중부경찰서,2018상
4,201801,차대차 - 기타,남,60대,남,50대,서울중부경찰서,2018상
...,...,...,...,...,...,...,...,...
140949,201908,차대사람 - 횡단중,남,40대,여,90대 이상,마산중부경찰서,2019하
140950,202012,차대사람 - 차도통행중,여,40대,여,90대 이상,창원중부경찰서,2020
140951,201904,차대사람 - 횡단중,남,60대,남,90대 이상,진해경찰서,2019상
140952,201905,차대사람 - 차도통행중,남,20대,남,90대 이상,진해경찰서,2019상


In [141]:
car_ac = car_ac.groupby(['jur_stn','year'])[['acc_case']].count()
car_ac

acc_case
jur_stn year           
마산동부경찰서 2018상       203
        2018하       186
        2019상       213
        2019하       243
        2020        382
...                 ...
창원중부경찰서 2018상       338
        2018하       349
        2019상       315
        2019하       337
        2020        555

[205 rows x 1 columns]

In [145]:
car_ac.loc[(slice(None),['2018상','2018하','2019상','2019하']),'acc_case'] = round(car_ac.loc[(slice(None),['2018상','2018하','2019상','2019하']),'acc_case']/6)
car_ac.loc[(slice(None),['2020']),'acc_case'] = round(car_ac.loc[(slice(None),['2020']),'acc_case']/12)


In [146]:
car_ac = car_ac.astype(int)
car_ac

acc_case
jur_stn year           
마산동부경찰서 2018상        34
        2018하        31
        2019상        36
        2019하        40
        2020         32
...                 ...
창원중부경찰서 2018상        56
        2018하        58
        2019상        52
        2019하        56
        2020         46

[205 rows x 1 columns]

# 경찰서 및 경찰 수 데이터 전처리

In [212]:
police

,nat_pol_agency,pol_office,sec_center,num
0,서울경찰청,서울중부경찰서,을지지구대,42
1,서울경찰청,서울중부경찰서,광희지구대,47
2,서울경찰청,서울중부경찰서,약수지구대,42
3,서울경찰청,서울중부경찰서,신당파출소,26
4,서울경찰청,서울중부경찰서,장충파출소,26
...,...,...,...,...
315,경남경찰청,진주경찰서,일반성파출소,13
316,경남경찰청,진주경찰서,지수파출소,9
317,경남경찰청,진주경찰서,수곡파출소,9
318,경남경찰청,진주경찰서,대곡파출소,9


In [213]:
police.loc[police['pol_office'].isin([ '서울로3경찰서', '서울류지경찰서', '서울리봉경찰서', '서울봉지경찰서', '서울왕파경찰서',
       '서울일지경찰서', '서울척지경찰서']) == True,'pol_office'] = '서울구로경찰서'

In [214]:
pol_num = police.groupby(['pol_office'])[['sec_center']].count()
pol_num

,sec_center
pol_office,
마산동부경찰서,7
마산중부경찰서,8
서울강남경찰서,8
서울강동경찰서,9
서울강북경찰서,9
서울강서경찰서,10
서울관악경찰서,9
서울광진경찰서,10
서울구로경찰서,8


In [215]:
pol_man = police.groupby(['pol_office'])[['num']].sum()
pol_man

,num
pol_office,
마산동부경찰서,185
마산중부경찰서,142
서울강남경찰서,421
서울강동경찰서,393
서울강북경찰서,347
서울강서경찰서,500
서울관악경찰서,493
서울광진경찰서,380
서울구로경찰서,395


In [216]:
police = pol_num.join(pol_man,how='inner')
police

,sec_center,num
pol_office,,
마산동부경찰서,7,185
마산중부경찰서,8,142
서울강남경찰서,8,421
서울강동경찰서,9,393
서울강북경찰서,9,347
서울강서경찰서,10,500
서울관악경찰서,9,493
서울광진경찰서,10,380
서울구로경찰서,8,395


In [217]:
police = police.rename({'sec_center':'경찰서 수','num':'경찰 수'},axis=1)
police

,경찰서 수,경찰 수
pol_office,,
마산동부경찰서,7,185
마산중부경찰서,8,142
서울강남경찰서,8,421
서울강동경찰서,9,393
서울강북경찰서,9,347
서울강서경찰서,10,500
서울관악경찰서,9,493
서울광진경찰서,10,380
서울구로경찰서,8,395


In [140]:
pip install area

You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [160]:
import geopandas as gpd
import json
from area import area

In [205]:
with open('1.5개시도_경찰서_관할경계.geojson', 'r') as f:
    pol_json = json.load(f)

# pol_area = json.dumps(json_data,ensure_ascii = False

In [206]:
pol_json

{'type': 'FeatureCollection',
 'name': '2',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'NAME': '세종경찰서', 'PNAME': '충남청'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[127.17202198487445, 36.73105976246728],
       [127.17202198487445, 36.731059762467275],
       [127.17200391076662, 36.731130430685994],
       [127.1720117799115, 36.73113172584495],
       [127.17202198487445, 36.73105976246728]]],
     [[[127.16607222376723, 36.73349166900014],
       [127.16616538907823, 36.73346899582924],
       [127.16649351598134, 36.73347036558834],
       [127.16659383763631, 36.733470290580556],
       [127.16659734137278, 36.73340673980766],
       [127.16659106077606, 36.73328570224651],
       [127.16658962801195, 36.73324355047704],
       [127.16658550516385, 36.73316821403404],
       [127.16656886327564, 36.73311795502324],
       [127.16654009922144, 36.73306885740821],
      

In [208]:
pol_area = pd.DataFrame(columns=['pol_office','면적'])
for i, x in enumerate(pol_json['features']):
    pol_area.loc[i,'pol_office'] = x['properties']['NAME']
    pol_area.loc[i,'면적'] = area(x['geometry'])/1000000

pol_area = pol_area.set_index('pol_office')
pol_area

,면적
pol_office,
세종경찰서,465.686
진주경찰서,714.142
창원서부경찰서,196.735
창원중부경찰서,96.0286
마산동부경찰서,92.3382
마산중부경찰서,234.701
진해경찰서,121.682
서울강동경찰서,23.4465
서울송파경찰서,30.8019


In [218]:
police = police.join(pol_area,how='inner')
police

,경찰서 수,경찰 수,면적
pol_office,,,
마산동부경찰서,7,185,92.3382
마산중부경찰서,8,142,234.701
서울강남경찰서,8,421,12.165
서울강동경찰서,9,393,23.4465
서울강북경찰서,9,347,23.693
서울강서경찰서,10,500,38.2984
서울관악경찰서,9,493,29.5681
서울광진경찰서,10,380,21.9023
서울구로경찰서,8,395,20.1231


In [219]:
police['경찰서 수/면적'] = police['경찰서 수'] / police['면적']
police['경찰 수/면적'] = police['경찰 수'] / police['면적']
police

,경찰서 수,경찰 수,면적,경찰서 수/면적,경찰 수/면적
pol_office,,,,,
마산동부경찰서,7,185,92.3382,0.0758082,2.0035
마산중부경찰서,8,142,234.701,0.034086,0.605026
서울강남경찰서,8,421,12.165,0.657625,34.6075
서울강동경찰서,9,393,23.4465,0.383852,16.7615
서울강북경찰서,9,347,23.693,0.379859,14.6457
서울강서경찰서,10,500,38.2984,0.261108,13.0554
서울관악경찰서,9,493,29.5681,0.304382,16.6734
서울광진경찰서,10,380,21.9023,0.456573,17.3498
서울구로경찰서,8,395,20.1231,0.397553,19.6292


In [132]:
data_112.to_csv('112신고_4.csv')
crime1.to_csv('범죄발생_3.csv')
crime2.to_csv('범죄검거_3.csv')
car_ac.to_csv('교통사고.csv')
police.to_csv('경찰서_경찰_수.csv')